# Examen final

## Contexte

Vous êtes analyste de données dans une entreprise qui souhaite analyser ses performances de ventes trimestrielles. On vous donne un dossier contenant 10 fichiers Excel. Chaque fichier comporte: 
- Date: Date à laquelle l'information a été enregistrée 
- Produit: Type de produit vendu; Il existe trois types de produit: A, B et C
- Région: Région dans laquelle la vente à eue lieu: Ouest, Sud, Est et Nord
- Quantité: Quantité vendue du produit
- Prix unitaire: Prix auquel une unité de produit est vendue
- Chiffre d'Affaire: Quantité * prix unitaire

Vous avez aussi accès à un fichier "référentiel.xlsx". Ce fichier comporte le type de produit et le prix unitaire de base de chaque produit. 

## Objectifs de l'examen

### Tronc commun (/13)

* **Tâches à effectuer** : 
1. Importez les dix fichiers Excels et fusionner les données en un seul DataFrame nommé "data" (/1)
2. Changez le type de donnée de la colonne "Date" en "datetime" (/1)
3. Vérifiez la présence de donnée manquante. Si donnée manquante il y a, remplacez la par la valeur qui la précède. (/1)
4. Calculez le chiffre d'affaire total et les chiffres d'affaires par produit. Stockez ces valeurs dans un DataFrame nommé "CA". (/1)
5. Créez une pivot table qui montre le chiffre d'affaire par mois (lignes) et par région (colonnes). (/2)
6. Importez le fichier référentiel et effectuez une jointure avec votre DataFrame "data". (/2)
7. Créez une nouvelle colonne qui représente les mois de ventes. Appelez la colonne "mois". (/1)
8. Créez un graphique qui présente l'évolution du chiffre d'affaire mensuel pour les trois produits. (/1)
9. Créez un "boxplot" des prix unitaires de chaque produits. (/1)
10. En utilisant xlwings, créez un nouveau fichier Excel nommé "Analyse_CA.xlsx" qui comporte (/2):
    - Spreadsheet 1: Le DataFrame "data"
    - Spreadsheet 2: Les graphiques boxplot et chiffre d'affaire mensuels
    - Spreadsheet 3: Le DataFrame "CA" et la pivot table (question 5.)

### Tronc avancé (/7): 

Pour ce tronc avancé, le détail des tâches à effectuer n'est pas disponible. C'est à vous de comprendre le problème et de le traduire en une série d'opérations en code python. Pour cela, vous allez devoir vous appuyer sur de la documentation. 

* L'entreprise veut stabiliser ces ventes sur l'année et pour cela et supprimer de son offre les produits dont la vente est trop irrégulière : **Identifiez les produits dont la vente est la plus fluctuante**. (/2)
**[Documentation 1](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.var.html)**, **[Documentation 2](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.std.html)**


* L'entreprise veut savoir si les prix qu'elle applique ont une incidence sur les quantités vendues. **Trouvez l'influence des prix sur les quantités vendues**. (/3) (Indice: Une régression linéaire peut permettre de trouver une telle relation) 
**[Documentation 3](https://www.statsmodels.org/stable/regression.html)**

* L'entreprise pensent qu'il existe un effet de saisonalité dans ses ventes. Particulièrement, il semblerait que les ventes du premier trimestre sont plus basses que les ventes du dernier trimestre.**Testez si cette différence est significative**. (/2)
**[Documentation 4](https://www.statsmodels.org/stable/generated/statsmodels.stats.weightstats.ttest_ind.html)**

### Fichiers attendus: 
1. Le notebook python qui effectue les 10 tâches à effectuer (Tronc commun) + la réponse aux questions ci-desus (Tronc avancé) **avec les explications nécessaires en Markdown**.
2. Le fichier Excel "Analyse_CA.xlsx"

*PS: 1pt bonus pour la clarté du code, la qualité du fichier excel et le design des graphiques!*

## Devoir réalisé par Alexandre, Adnan, Sonia, Elisa

## 1 - PLAN D'ACTION

### Importer les librairies nécessaires
- Utiliser Pandas pour traiter les données excel
- Utiliser matplot pour faire des analyses

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import os

### Regrouper les tableurs en un seul DataFrame
- Trouver le chemin du répertoire des tableurs
- Créer une liste auquel on ajoutera le nom de tous les fichiers avec une boucle for et la méthode append()
- Utiliser la methode concat() pour fusionner les tableurs

In [10]:
# Chemin du répertoire contenant les tableurs à fusionner
repertoire = "/Users/Harry/Desktop/examen_data"

# Liste pour stocker les noms de fichiers à fusionner
fichiers_a_fusionner = []

# Parcourir les fichiers du répertoire et ajouter les noms des fichiers Excel à la liste
for fichier in os.listdir(repertoire):
    if fichier.endswith(".xlsx"):
        fichiers_a_fusionner.append(os.path.join(repertoire, fichier))

# Créer un DataFrame vide pour stocker les données fusionnées
fusion = pd.DataFrame()

# Boucle pour lire chaque fichier Excel et fusionner ses données dans le DataFrame fusion
for fichier in fichiers_a_fusionner:
    data = pd.read_excel(fichier)
    fusion = pd.concat([fusion, data], ignore_index=True)

# Enregistrer le DataFrame fusionné dans un nouveau fichier Excel
fusion.to_excel("data.xlsx", index=False)

fusion.head()

,Date,Produit,Région,Quantité,Prix unitaire,Chiffre d'affaires,Catégorie,Prix Standard
0,2023-10-01,Produit B,Est,19.0,22.710119,431.492259,NaN,NaN
1,2023-10-21,Produit A,Ouest,17.0,61.378968,1043.442448,NaN,NaN
2,2023-08-15,Produit A,Est,6.0,26.674332,160.045992,NaN,NaN
3,2023-01-15,Produit B,Est,16.0,35.077876,561.246011,NaN,NaN
4,2023-07-11,Produit B,Est,12.0,29.683882,356.206584,NaN,NaN


### Changer le type de donnée de la colonne "Date"
- Utiliser la méthode to_datetime()
- Vérifier que le type a bien changé

In [9]:
fusion['Date'] = pd.to_datetime(fusion['Date'])

print(fusion['Date'].dtype)

datetime64[ns]


### Vérifier les données manquantes
- Utiliser une boucle for pour vérifier si la cellule est vide, si elle est vide la remplacer par la précédente

In [18]:
# Remplacer les cellules vides par la valeur de la ligne précédente
fusion.ffill(inplace=True)

fusion

,Date,Produit,Région,Quantité,Prix unitaire,Chiffre d'affaires,Catégorie,Prix Standard
0,2023-10-01,Produit B,Est,19.0,22.710119,431.492259,NaN,NaN
1,2023-10-21,Produit A,Ouest,17.0,61.378968,1043.442448,NaN,NaN
2,2023-08-15,Produit A,Est,6.0,26.674332,160.045992,NaN,NaN
3,2023-01-15,Produit B,Est,16.0,35.077876,561.246011,NaN,NaN
4,2023-07-11,Produit B,Est,12.0,29.683882,356.206584,NaN,NaN
...,...,...,...,...,...,...,...,...
998,2023-08-09,Produit C,Est,15.0,15.990826,239.862383,Alimentation,15.0
999,2023-08-23,Produit B,Sud,5.0,68.348621,341.743103,Alimentation,15.0
1000,2023-11-27,Produit C,Nord,17.0,16.669488,283.381297,Alimentation,15.0
1001,2023-05-26,Produit C,Sud,6.0,43.792204,262.753221,Alimentation,15.0


### Calculer le chiffre d'affaire total et par produit
- Créer une catégorie "produit" avec la méthode groupby
- Faire une somme des chiffres d'affaires par produit et de la colonne "Chiffre d'affaires" avec sum

In [23]:
# Création de la catégorie Produit
product_type = fusion.groupby('Produit')

In [28]:
# Sommer le chiffre d'affaires par produit et total
product_turnover = product_type["Chiffre d'affaires"].sum()
total_turnover = fusion["Chiffre d'affaires"].sum()

print(product_turnover, total_turnover)

Produit
Produit A    187136.728911
Produit B    180028.643481
Produit C    179391.899070
Name: Chiffre d'affaires, dtype: float64
546557.2714610326


In [45]:
CA = pd.DataFrame(product_turnover)
CA.loc['Total'] = total_turnover
CA.head()

,Chiffre d'affaires
Produit,
Produit A,187136.728911
Produit B,180028.643481
Produit C,179391.899070
Total,546557.271461


### Créez une pivot table pour le CA mensuel et par région
- Utiliser la  méthode pivot_table
- Créer une nouvelle colonne en extrayant le mois de la colonne date

In [49]:
# Créer une nouvelle colonne 'Mois' pour extraire le mois à partir de la colonne 'date'
fusion['Mois'] = fusion['Date'].dt.month

# Création d'une pivot table

CA_pivot_table = pd.pivot_table(fusion, values="Chiffre d'affaires", index=['Mois','Région'] , aggfunc='sum')
print(CA_pivot_table)

             Chiffre d'affaires
Mois Région                    
1    Est           10037.028399
     Nord          13076.338478
     Ouest          7710.564014
     Sud            8213.807744
2    Est            9837.787233
     Nord           9853.258419
     Ouest         19094.963887
     Sud           10051.177005
3    Est            3562.719474
     Nord           8653.392696
     Ouest         10167.234999
     Sud            8646.945490
4    Est            9297.540059
     Nord          14065.944384
     Ouest         10668.264692
     Sud           14607.472470
5    Est           10775.295424
     Nord          16075.861928
     Ouest         13966.637315
     Sud            8783.433772
6    Est           12085.008995
     Nord          12739.880874
     Ouest         12484.007879
     Sud           12037.267731
7    Est            9123.243007
     Nord          10750.532192
     Ouest         10123.820477
     Sud           10406.780030
8    Est           16581.733330
     Nor

### Ajouter le référentiel 
- Utiliser la méthode join()

In [56]:
referentiel = pd.read_excel('/Users/Harry/Desktop/examen_data/referentiel.xlsx')

AttributeError: 'DataFrame' object has no attribute 'concat'